## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [32]:
import tqdm 
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 8.55 ms


### Part 0: Download the TED dataset

In [21]:
import urllib.request
import zipfile
import lxml.etree

time: 2.63 ms


In [22]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

time: 3.37 ms


In [23]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

time: 1.36 s


### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [24]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

time: 3.19 ms


Let's start by removing all parenthesized strings using a regex:

In [25]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

time: 76.9 ms


We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [26]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

time: 8.68 ms


Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [27]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

time: 303 ms


Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [28]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

time: 2.46 s


Two sample processed sentences:

In [29]:
len(sentences_ted)

266694

time: 2.17 ms


In [30]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
time: 1.78 ms


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [35]:
from collections import Counter
ted_word_counts = [Counter(sentence_list) for sentence_list in sentences_ted]
final_counts = Counter()
for ted_count in tqdm.tqdm(ted_word_counts):
    final_counts += ted_count


100%|██████████| 266694/266694 [09:40<00:00, 459.52it/s]

time: 9min 42s


In [36]:
import operator
len(final_counts.keys())
sorted_final_counts = sorted(final_counts.items(), key=operator.itemgetter(1))
sorted_final_counts.reverse()
counts_ted_top1000 = np.array([value for _,value in sorted_final_counts[:1000]])
words_top_ted = [key for key,_ in sorted_final_counts[:1000]]

time: 56.4 ms


Plot distribution of top-1000 words

In [37]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

time: 90.2 ms


### Part 3: Train Word2Vec

In [38]:
from gensim.models import Word2Vec

time: 338 ms


In [39]:
model = Word2Vec(sentences_ted, size=100, window=5, min_count=10, workers=4)
model.save("ted_word2vec.model")

time: 10.7 s


In [40]:
model_ted = Word2Vec.load("ted_word2vec.model")

time: 103 ms


### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [41]:
model_ted.most_similar("man")

[('woman', 0.8497589826583862),
 ('guy', 0.7974798679351807),
 ('lady', 0.7795628905296326),
 ('boy', 0.7521253228187561),
 ('gentleman', 0.7463899850845337),
 ('girl', 0.7416039109230042),
 ('poet', 0.6975832581520081),
 ('kid', 0.6701671481132507),
 ('soldier', 0.6684935092926025),
 ('david', 0.6652417182922363)]

time: 14.1 ms


In [42]:
model_ted.most_similar("computer")

[('machine', 0.7700843811035156),
 ('software', 0.7199214696884155),
 ('robot', 0.7062963843345642),
 ('device', 0.6655927896499634),
 ('3d', 0.6401070356369019),
 ('program', 0.6369738578796387),
 ('interface', 0.6364388465881348),
 ('simulation', 0.6358679533004761),
 ('camera', 0.6300681233406067),
 ('video', 0.6297894716262817)]

time: 10.1 ms


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [43]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

time: 10.1 ms


In [44]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

time: 3.99 s


In [45]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

time: 52.3 ms


### Part 5: Wiki Learnt Representations

Download dataset

In [46]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

time: 2.86 ms


In [47]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

time: 2.73 s


Preprocess sentences (note that it's important to remove small sentences for performance)

In [48]:
sentences_strings_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_strings_wiki.extend(s)
    
for s_i in tqdm.tqdm(range(len(sentences_strings_wiki))):
    sentences_strings_wiki[s_i] = re.sub("[^a-z]", " ", sentences_strings_wiki[s_i].lower())
    sentences_strings_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_strings_wiki[s_i])
del input_text

100%|██████████| 4267112/4267112 [00:37<00:00, 113333.64it/s]

time: 46.2 s


In [49]:
# sample 1/5 of the data
shuffle(sentences_strings_wiki)
print(len(sentences_strings_wiki))


4267112
time: 4.19 s


In [50]:
sentences_strings_wiki = sentences_strings_wiki[:int(len(sentences_strings_wiki)/5)]
print(len(sentences_strings_wiki))


853422
time: 294 ms


In [51]:
sentences_wiki = []
for sent_str in tqdm.tqdm(sentences_strings_wiki):
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_wiki.append(tokens)

100%|██████████| 853422/853422 [00:09<00:00, 87265.19it/s] 

time: 9.79 s


In [52]:
print (sentences_wiki[0])
print (sentences_wiki[1])

['first', 'steps']
['in', 'games', 'with', 'the', 'netherlands', 'he', 'recorded', 'a']
time: 2.87 ms


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [53]:
%time
wiki_word_counts = [Counter(sentence_list) for sentence_list in sentences_wiki]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.9 µs
time: 6.49 s


In [54]:
final_counts_wiki = Counter()
print (len(wiki_word_counts))


853422
time: 952 µs


In [55]:
%time
for wiki_count in tqdm.tqdm(wiki_word_counts):
    final_counts_wiki += wiki_count

  0%|          | 480/853422 [00:00<02:57, 4795.89it/s]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.6 µs


100%|██████████| 853422/853422 [4:04:04<00:00, 58.28it/s]  

time: 4h 4min 4s


In [56]:
print (len(final_counts_wiki.keys()))
sorted_final_counts_wiki = sorted(final_counts_wiki.items(), key=operator.itemgetter(1))
sorted_final_counts_wiki.reverse()
counts_wiki_top1000 = np.array([value for _,value in sorted_final_counts_wiki[:1000]])
words_top_wiki = [key for key,_ in sorted_final_counts_wiki[:1000]]

234948
time: 146 ms


In [57]:
model1 = Word2Vec(sentences_wiki, size=100, window=5, min_count=10, workers=4)
model1.save("wiki_word2vec.model")
model_wiki = Word2Vec.load("wiki_word2vec.model")

time: 48.6 s


In [59]:
words_top_wiki[0]

'the'

time: 4.1 ms


In [60]:
model_wiki.most_similar("man")

[('woman', 0.7459496259689331),
 ('person', 0.6308660507202148),
 ('girl', 0.6241695880889893),
 ('boy', 0.5806270837783813),
 ('soldier', 0.5727490186691284),
 ('someone', 0.543090283870697),
 ('kid', 0.5382193922996521),
 ('dog', 0.5380415916442871),
 ('thief', 0.5373824834823608),
 ('knife', 0.5320389270782471)]

time: 30.9 ms


In [61]:
model_wiki.most_similar("computer")

[('software', 0.7582915425300598),
 ('user', 0.7185367345809937),
 ('simulation', 0.7015782594680786),
 ('browser', 0.6973167657852173),
 ('hardware', 0.6929763555526733),
 ('computing', 0.6904938220977783),
 ('graphical', 0.6893253922462463),
 ('sensor', 0.6835991144180298),
 ('computers', 0.6835297346115112),
 ('interface', 0.6821520328521729)]

time: 11.6 ms


#### t-SNE visualization

In [62]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

time: 3.93 s


In [63]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

time: 115 ms
